In [58]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb
%run /home/dbaciur/NTU/NTU/notes/CommonUtils.ipynb

Constants loaded


In [32]:
import torch
import torchvision as tv

import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import intervals as I
import moviepy
from pathlib import Path
from moviepy.editor import VideoFileClip
import pandas as pd
import json as j

### Begin frames dir

In [42]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_PATH, converters=C.F_CONVERTERS)
frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout,Splits,Origin
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__3435.269__40__B_CHAR...,point event
1,PH1011-PHYSICS_20150922__3703.298__43__B_CHART...,PH1011-PHYSICS_20150922,3703.298,43,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__3703.298__43__B_CHAR...,point event
2,PH1011-PHYSICS_20150922__4040.803__44__B_CHART...,PH1011-PHYSICS_20150922,4040.803,44,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4040.803__44__B_CHAR...,point event
3,PH1011-PHYSICS_20150922__4284.434__48__B_CHART...,PH1011-PHYSICS_20150922,4284.434,48,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4284.434__48__B_CHAR...,point event
4,PH1011-PHYSICS_20150922__4957.804__62__B_CHART...,PH1011-PHYSICS_20150922,4957.804,62,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4957.804__62__B_CHAR...,point event


In [43]:
Path(C.Frames.MAIN_DIR).mkdir(exist_ok=True)
frames_df.to_csv(C.Frames.POINT_DUPLICATES, index=False)
len(frames_df)

4920

### Load frames

In [44]:
frames_df = pd.read_csv(C.Frames.POINT_DUPLICATES, converters=C.F_CONVERTERS)
frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout,Splits,Origin
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__3435.269__40__B_CHAR...,point event
1,PH1011-PHYSICS_20150922__3703.298__43__B_CHART...,PH1011-PHYSICS_20150922,3703.298,43,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__3703.298__43__B_CHAR...,point event
2,PH1011-PHYSICS_20150922__4040.803__44__B_CHART...,PH1011-PHYSICS_20150922,4040.803,44,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4040.803__44__B_CHAR...,point event
3,PH1011-PHYSICS_20150922__4284.434__48__B_CHART...,PH1011-PHYSICS_20150922,4284.434,48,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4284.434__48__B_CHAR...,point event
4,PH1011-PHYSICS_20150922__4957.804__62__B_CHART...,PH1011-PHYSICS_20150922,4957.804,62,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__4957.804__62__B_CHAR...,point event


### Load events & frames

In [45]:
events_df = pd.read_csv(C.VISUAL_EVENTS_PATH, converters=C.A_CONVERTERS)
events_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time,Start H (s),Stop H (s),Time H
0,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.103,00:02:12:351,00:47:51:855,00:25:02:103


In [46]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_PATH, converters=C.F_CONVERTERS)
frames_df.head(1)

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layouts,Layout,Splits,Origin
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]","[1, 1]",3.0,[PH1011-PHYSICS_20150922__3435.269__40__B_CHAR...,point event


### Get time for already existing point frames

In [24]:
websites_point_frames = frames_df[frames_df[C.F_BEH_ID] == C.B_WEBSITE_P]

# result
video_to_websites_point_times = {}

for _, frame in websites_point_frames.iterrows():
    
    time = frame[C.F_TIME]
    video = frame[C.F_VIDEO]
    
    if video not in video_to_websites_point_times:
        video_to_websites_point_times[video] = []
    video_to_websites_point_times[video].append(time)
    
len(video_to_websites_point_times)
print(j.dumps(video_to_websites_point_times, indent=2))

{
  "15S1-PH1105-LEC_20150824": [
    1829.52,
    1966.897,
    1794.874,
    1825.623,
    2473.994,
    2492.246
  ],
  "15S1-PH1105-LEC_20150828": [
    2575.476,
    2779.287000000001,
    8.857000000000003
  ],
  "15S1-MH1800-LEC_20150817": [
    434.346,
    472.847,
    495.097,
    871.159,
    901.658,
    954.907,
    430.391,
    454.518,
    486.811,
    868.298,
    900.548,
    958.543,
    1261.058,
    5906.526,
    5922.026999999999,
    6175.025,
    433.01,
    869.3560000000001,
    957.417
  ],
  "15S1-MH1800-LEC_20150824": [
    3241.443,
    3993.035,
    71.872,
    3224.942,
    3984.858,
    4210.833,
    4578.67,
    4586.4209999999985
  ],
  "15S2-PH1106-LEC_20160113": [
    756.248,
    1018.09,
    1019.716,
    1029.712,
    1295.4589999999996,
    763.184,
    1025.142
  ],
  "15S2-PH1106-LEC_20160127": [
    541.725,
    6097.224,
    6866.694,
    6947.408,
    533.6030000000002,
    543.345
  ],
  "15S2-PH1106-LEC_20160210": [
    6547.766
  ],
  "16

### Check event layout

In [25]:
video_durations = {}

for video in tqdm(frames_df[C.F_VIDEO].unique()):
    
    video_path = f"{C.ORIGINAL_VIDEOS_DIR}/{video}.mp4"
    video_clip = VideoFileClip(video_path)
    
    video_durations[video] = video_clip.duration

In [26]:
video_layout_annotations = {}

for video in tqdm(frames_df[C.F_VIDEO].unique()):    
    layouts_torch = torch.load(f"{C.LAYOUT_ANNOTATIONS_DIR}/{video}.pt")
    video_layout_annotations[video] = layouts_torch.numpy()

In [27]:
def get_timestmap_layout(video, time):
    
    layout_annotations = video_layout_annotations[video]
    layout_annotations_size = len(layout_annotations)
    
    video_duration = video_durations[video]
    layout_index_approximation = (time / video_duration) * layout_annotations_size
    layout_index_approximation = int(layout_index_approximation)
    layout = layout_annotations[layout_index_approximation]
    layout = int(layout)
    
    return layout

def is_screen_layout(layout):
    return layout in [2, 4, 5, 6, 8, 9, 10]

def get_split_layouts(layout):
    # layout < 3 => whole screen is either Camera or Screen
    if layout < 3:
        layouts = [layout, layout]
        
    # layout < 7 => layout is divided into 3 parts:
    # - 1 vertical line in 1/3 of width middle that splits screen to L and R
    # - 1 horizontal line in the middle of L
    elif layout < 7:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 3]
    # layout >=7 => layout is split in half
    else:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 7]
    
    return layouts

def get_splits(frame_id, layout):
    if layout < 3:
        return [f"{frame_id}__Full", f"{frame_id}__Full"]
    
    return [f"{frame_id}__Left", f"{frame_id}__Right"]

### 1st approach (proabably invalid)

Divide every state website event into 10s chunks and add frame at the beggining of each one IF there's not already
some point event in distance <10s.

In [65]:
# take frames distant from each other for around 1 second
FRAMES_DISTANCE = 5

frames_added = 0
frames_omitted_by_distance = 0
frames_omitted_by_layout = 0
video_timestamps_omitted_by_layout = []

df_data = []

for video in tqdm(frames_df[C.F_VIDEO].unique()):
    
    websites_state_events = events_df[events_df[C.A_VIDEO] == video]
    websites_state_events = websites_state_events[websites_state_events[C.A_BEH_ID] == C.B_WEBSITE_S]
    
    frame_times_for_video = []
    frame_times_for_video.extend(video_to_websites_point_times.get(video, []))
    
    for _, event in websites_state_events.iterrows():
        
        start = event[C.A_START]
        stop = event[C.A_STOP]
        event_id = event[C.A_EVENT_ID]
        beh_id = event[C.A_BEH_ID]
        ra_id = event[C.A_RA_ID]
        
        for time_candidate in np.arange(start, stop, FRAMES_DISTANCE):
            
            frame_id = f"{video}__{time_candidate}__{event_id}__{beh_id}__{ra_id}"
            
            layout = get_timestmap_layout(video, time_candidate)
            layouts = get_split_layouts(layout)
            splits = get_splits(frame_id, layout)
                
            is_too_close_to_other_frame = False
            for other_time in frame_times_for_video:
                if abs(time_candidate - other_time) < FRAMES_DISTANCE:
                    is_too_close_to_other_frame = True
#                     print(f"{video}, {time_candidate}, {other_time}, {layout}, [{start}-{stop}]")
                    break
                
            if is_too_close_to_other_frame:
                frames_omitted_by_distance = frames_omitted_by_distance + 1
                continue
            
            if not is_screen_layout(layout):
                frames_omitted_by_layout = frames_omitted_by_layout + 1
                video_timestamps_omitted_by_layout.append((video, format_seconds(time_candidate)))
#                 print(f"bad layout {time_candidate}, {layout}")
                continue
              
            frames_added = frames_added + 1
            df_data.append([
                frame_id,
                video,
                time_candidate,
                event_id,
                beh_id,
                ra_id,
                [], # collision events = deliberately empty for now
                [], # behs = deliberately empty for now
                layout,
                layouts,
                splits,
                C.F_ORIG_WEBSITES_STATE_5
            ])
            
print(frames_added)
print(frames_omitted_by_distance)
print(frames_omitted_by_layout)


4990
386
819


In [48]:
video_timestamps_omitted_by_layout

[('16S1-PH1801-LEC_20161020', '00:24:33:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:38:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:43:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:48:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:53:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:58:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:03:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:08:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:13:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:18:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:23:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:28:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:33:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:38:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:43:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:48:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:53:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:58:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:03:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:08:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:13:022'),
 ('16S1-PH180

In [49]:
new_frames_df = pd.DataFrame(
        data=df_data,
        columns=[
            C.F_ID,
            C.F_VIDEO,
            C.F_TIME,
            C.F_EVENT_ID,
            C.F_BEH_ID,
            C.F_RA_ID,
            C.F_COLL_EVENTS,
            C.F_BEHS,
            C.F_LAYOUT,
            C.F_LAYOUTS,
            C.F_SPLITS,
            C.F_ORIGIN,
        ]
    )

In [50]:
new_frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layout,Layouts,Splits,Origin
0,15S1-PH1105-LEC_20150824__1788.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1788.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1788.769__1909__B_W...,websites state generated 5s
1,15S1-PH1105-LEC_20150824__1803.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1803.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1803.769__1909__B_W...,websites state generated 5s
2,15S1-PH1105-LEC_20150824__1808.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1808.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1808.769__1909__B_W...,websites state generated 5s
3,15S1-PH1105-LEC_20150824__1813.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1813.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1813.769__1909__B_W...,websites state generated 5s
4,15S1-PH1105-LEC_20150824__1818.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1818.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1818.769__1909__B_W...,websites state generated 5s


In [51]:
len(new_frames_df)

4990

In [59]:
new_frames_df.to_csv(C.Frames.WEBSITE_S10_DUPLICATES, index=False)

### 2nd approach

Take into consideration a fact, that there are 3 RAs, so even though all frames are not colliding with point events,
if 2RAs annotated 2 state events having same time range - all new frames would be duplicated.
Also - remove duplicates in point events.

In [60]:
website_df = pd.read_csv(C.Frames.WEBSITE_S10_DUPLICATES, converters=C.F_CONVERTERS)
website_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layout,Layouts,Splits,Origin
0,15S1-PH1105-LEC_20150824__1788.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1788.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1788.769__1909__B_W...,websites state generated 5s
1,15S1-PH1105-LEC_20150824__1803.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1803.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1803.769__1909__B_W...,websites state generated 5s
2,15S1-PH1105-LEC_20150824__1808.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1808.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1808.769__1909__B_W...,websites state generated 5s
3,15S1-PH1105-LEC_20150824__1813.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1813.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1813.769__1909__B_W...,websites state generated 5s
4,15S1-PH1105-LEC_20150824__1818.769__1909__B_WE...,15S1-PH1105-LEC_20150824,1818.769,1909,B_WEBSITE_S,RA1,[],[],4,"[1, 2]",[15S1-PH1105-LEC_20150824__1818.769__1909__B_W...,websites state generated 5s


In [61]:
frames_without_collision = []
frames_with_collision = []

for video in tqdm(website_df[C.F_VIDEO].unique()):
    
    video_website_df = website_df[website_df[C.F_VIDEO] == video]
    occupied_timestamps = []
    
    for _, frame in video_website_df.iterrows():
        
        time = frame[C.F_TIME]
        
        has_collision = False
        for t in occupied_timestamps:
            if abs(t - time) < 10:
                has_collision = True
                break
                
        if not has_collision:
            frames_without_collision.append(frame[C.F_ID])
            occupied_timestamps.append(time)
        else:
            frames_with_collision.append(frame[C.F_ID])

In [62]:
print(len(frames_without_collision))
print(len(frames_with_collision))

1452
3538


In [63]:
less_website_df = website_df[website_df[C.F_ID].isin(frames_without_collision)]
len(less_website_df)

1452

In [64]:
less_website_df.to_csv(C.Frames.WEBSITE_S10, index=False)